In [ ]:
import glob 
import os
from PIL import Image
from PIL import ImageDraw
from PIL import ImageOps
from PIL import ImageFont

In [ ]:
#creates surface figure for one contrast/group/timepoint

def create_subcortical_figure_by_group(group,ses,task,contrast):
    
    #check if file exists
    if len(glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_display-coronal.png'))==0:
        print(f'no data for group {group} session {ses} task {task} contrast {contrast}')
        return 
    
    #get coronal image and its dim
    coronal_path = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_display-coronal.png')[0]
    coronal=Image.open(coronal_path,mode='r')
    w_c,h_c=coronal.size
    
    #find area locations of the 4 brains + cbar
    area_coronal_1 = (0, 0, 219, h_c)
    area_coronal_2 = (219, 0, 438, h_c)
    area_coronal_3 = (438, 0, 657, h_c)
    area_coronal_4 = (657, 0, 876, h_c)
    area_coronal_cbar = (897, 0, w_c, h_c)

    coronal_crop_1 = coronal.crop(area_coronal_1)
    coronal_crop_2 = coronal.crop(area_coronal_2)
    coronal_crop_3 = coronal.crop(area_coronal_3)
    coronal_crop_4 = coronal.crop(area_coronal_4)
    coronal_crop_cbar = coronal.crop(area_coronal_cbar)

    #set up new image
    new_im = Image.new('RGB', ( (coronal_crop_1.size[0]*2+coronal_crop_cbar.size[0]) , coronal_crop_1.size[1]*2) ,(255, 255, 255, 1)) #make a new image

    #paste in the cut out images
    new_im.paste(coronal_crop_1,(0,0)) 
    new_im.paste(coronal_crop_2,(coronal_crop_1.size[0],0))
    new_im.paste(coronal_crop_cbar,(coronal_crop_1.size[0]*2,0))

    new_im.paste(coronal_crop_3,(0,coronal_crop_cbar.size[1])) 
    new_im.paste(coronal_crop_4,(coronal_crop_1.size[0],coronal_crop_cbar.size[1]))
    new_im.paste(coronal_crop_cbar,(coronal_crop_1.size[0]*2,coronal_crop_cbar.size[1]))


    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_surface','visualization','intermediate_cifti_figures')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)

    threshold = coronal_path.split('threshold-')[1].split('_')[0]
    part_count = coronal_path.split('_n-')[1].split('_')[0]

    new_im.save(f'../../../derivatives/task_analysis_surface/visualization/intermediate_cifti_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{threshold}_n-{part_count}_cifti_figure_subcortical.png')
    
    return


In [ ]:
#creates surface figure for one contrast/group/timepoint

def create_surf_figure_by_group(group,ses,task,contrast):
    area = (30, 60, 370, 330)
    
    if len(glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_display-righthemi.png'))==0:
        print(f'no data for group {group} session {ses} task {task} contrast {contrast}')
        return 

    right_hemi_path = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_display-righthemi.png')[0]
    right_hemi=Image.open(right_hemi_path,mode='r')
    rH = right_hemi.crop(area) #crop just the part of the image that you want

    right_flat_path = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_display-rightflat.png')[0]
    right_flat=Image.open(right_flat_path,mode='r')
    rF = right_flat.crop(area) #crop just the part of the image that you want

    left_hemi_path = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_display-lefthemi.png')[0]
    left_hemi=Image.open(left_hemi_path,mode='r')
    lH = left_hemi.crop(area) #crop just the part of the image that you want

    left_flat_path = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_display-leftflat.png')[0]
    left_flat=Image.open(left_flat_path,mode='r')
    lF = left_flat.crop(area) #crop just the part of the image that you want
    lF = lF.transpose(Image.FLIP_LEFT_RIGHT)


    w,h=lF.size
    
    new_im = Image.new('RGB', ( (w*2) , h*2) ,(255, 255, 255, 1)) #make a new image
    #paste in the cut out images
    new_im.paste(lF,(0,h)) 
    new_im.paste(rF,(w,h))
    new_im.paste(lH,(0,0))
    new_im.paste(rH,(w,0))
    
    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_surface','visualization','intermediate_cifti_figures')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)
        
    threshold = right_hemi_path.split('threshold-')[1].split('_')[0]
    part_count = right_hemi_path.split('_n-')[1].split('_')[0]
    
    new_im.save(f'../../../derivatives/task_analysis_surface/visualization/intermediate_cifti_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{threshold}_n-{part_count}_cifti_figure_surface.png')
                
    return



In [ ]:
#stitches together surface and subcortical for one contrast/group/timepoint

def create_cifti_figure_by_group(group,ses,task,contrast):
    
    if len(glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_display-righthemi.png'))==0 or len(glob.glob(f'../../../derivatives/task_analysis_surface/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_display-coronal.png'))==0:
        print(f'no data for group {group} session {ses} task {task} contrast {contrast}')
        return (None,None)
    
    surface_path = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/intermediate_cifti_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_cifti_figure_surface.png')[0]
    surface = Image.open(surface_path,mode='r')
    w_surf, h_surf = surface.size
    
    subcortical_path = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/intermediate_cifti_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-*_n-*_cifti_figure_subcortical.png')[0]
    subcortical = Image.open(subcortical_path,mode='r')
    subcortical = ImageOps.contain(subcortical, (w_surf,w_surf))
    w_subcortical, h_subcortical = subcortical.size
    
    
    new_im = Image.new('RGB', ( w_subcortical , h_surf+h_subcortical) ,(255, 255, 255, 1)) #make a new image
    #paste in the cut out images
    new_im.paste(surface,(0,0)) 
    new_im.paste(subcortical,(0,h_surf))
    
    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_surface','visualization','intermediate_cifti_figures')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)

    threshold = surface_path.split('threshold-')[1].split('_')[0]
    part_count = surface_path.split('_n-')[1].split('_')[0]

    new_im.save(f'../../../derivatives/task_analysis_surface/visualization/intermediate_cifti_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{threshold}_n-{part_count}_cifti_figure_by_group.png')
    
    return (h_surf, h_subcortical)

In [ ]:
#stitch together the groups/timepoints for one contrast of nback or sst

def create_cifti_figure_by_contrast(task, contrast):

    png_paths = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/intermediate_cifti_figures/group-*_ses-*_*task-{task}_contrast-{contrast}_*_cifti_figure_by_group.png')
    HCb=Image.open(png_paths[0],mode='r')
    MMb=Image.open(png_paths[2],mode='r')
    MM1=Image.open(png_paths[1],mode='r')
    w,h = HCb.size

    new_im = Image.new('RGB', ( (w*3) , h) ,(255, 255, 255, 1)) #make a new image
    #paste in the cut out images
    new_im.paste(HCb,(0,0)) 
    new_im.paste(MMb,(w,0))
    new_im.paste(MM1,(w*2,0))
        
    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_surface','visualization','final_cifti_figures')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)

    new_im.save(f'../../../derivatives/task_analysis_surface/visualization/intermediate_cifti_figures/task-{task}_contrast-{contrast}_cifti_figure_by_contrast.png')

    return w


In [ ]:
#stitch together the various contrasts for a task

def create_surf_figure_all(task,contrasts,width_surf,height_surf,height_subcortical):
    png_paths = glob.glob(f'../../../derivatives/task_analysis_surface/visualization/intermediate_cifti_figures/task-{task}_contrast-*_cifti_figure_by_contrast.png')

    font = ImageFont.truetype("/usr/share/fonts/Liberation/LiberationMono-Bold.ttf",35)

    if task == 'nback':
        
        Twoback_Zeroback=Image.open(png_paths[0],mode='r')
        w,h=Twoback_Zeroback.size
        new_im = Image.new('RGB', ( w+300 , h+50) ,(255, 255, 255, 1)) #make a new image
        new_im.paste(Twoback_Zeroback,(300,50))
        
        draw = ImageDraw.Draw(new_im)
        
        draw.text((0.5*width_surf+300,0),"HC baseline",fill=(0,0,0),font=font,anchor="ma")
        draw.text((1.5*width_surf+300,0),"MCC baseline",fill=(0,0,0),font=font,anchor="ma")
        draw.text((2.5*width_surf+300,0),"MCC one-year",fill=(0,0,0),font=font,anchor="ma")

        draw.multiline_text((0,height_surf),"Two-back vs.\nZero-back",fill=(0,0,0),font=font,anchor="ls")

        display(new_im)
        
        new_im.save(f'../../../derivatives/task_analysis_surface/visualization/final_cifti_figures/task-{task}_contrast1-{contrasts[0]}_cifti_figure.png')

        return 
    
    if task == 'mid':
        Loss_AvoidLoss=Image.open(png_paths[0],mode='r')
        Win_NoWin=Image.open(png_paths[1],mode='r')
        w,h=Loss_AvoidLoss.size
        
        new_im = Image.new('RGB', ( w+300 , h*2+50) ,(255, 255, 255, 1)) #make a new image
        #paste in the cut out images
        new_im.paste(Win_NoWin,(300,50))
        new_im.paste(Loss_AvoidLoss,(300,h+50))
        
        draw = ImageDraw.Draw(new_im)
           
        draw.text((0.5*width_surf+300,0),"HC baseline",fill=(0,0,0),font=font,anchor="ma")
        draw.text((1.5*width_surf+300,0),"MCC baseline",fill=(0,0,0),font=font,anchor="ma")
        draw.text((2.5*width_surf+300,0),"MCC one-year",fill=(0,0,0),font=font,anchor="ma")
 
        draw.multiline_text((0,height_surf),"Reward vs.\nMissed Reward",fill=(0,0,0),font=font,anchor="ls")
        draw.multiline_text((0,height_surf*2+height_subcortical),"Loss vs.\nAvoided Loss",fill=(0,0,0),font=font,anchor="ls")
        
        display(new_im)
        
        new_im.save(f'../../../derivatives/task_analysis_surface/visualization/final_cifti_figures/task-{task}_contrast1-{contrasts[0]}_contrast2-{contrasts[1]}_cifti_figure.png')

    if task == 'sst':
        Succ_Go=Image.open(png_paths[0],mode='r')
        Unsecc_Go=Image.open(png_paths[1],mode='r')
        Unsecc_Succ=Image.open(png_paths[2],mode='r')
        w,h=Succ_Go.size
        
        new_im = Image.new('RGB', ( w+300 , h*3+50) ,(255, 255, 255, 1)) #make a new image
        #paste in the cut out images
        new_im.paste(Succ_Go,(300,50))
        new_im.paste(Unsecc_Go,(300,h+50))
        new_im.paste(Unsecc_Succ,(300,h*2+50))
        
        draw = ImageDraw.Draw(new_im)
        
        draw.text((0.5*width_surf+300,0),"HC baseline",fill=(0,0,0),font=font,anchor="ma")
        draw.text((1.5*width_surf+300,0),"MCC baseline",fill=(0,0,0),font=font,anchor="ma")
        draw.text((2.5*width_surf+300,0),"MCC one-year",fill=(0,0,0),font=font,anchor="ma")
 
        draw.multiline_text((0,height_surf),"Successful\nSTOP vs.\nGO",fill=(0,0,0),font=font,anchor="ls")
        draw.multiline_text((0,height_surf*2+height_subcortical),"Unsuccessful\nSTOP vs.\nGO",fill=(0,0,0),font=font,anchor="ls")
        draw.multiline_text((0,height_surf*3+height_subcortical*2),"Unsuccessful\nSTOP vs.\nSuccessful\nSTOP",fill=(0,0,0),font=font,anchor="ls")

        display(new_im)
        
        new_im.save(f'../../../derivatives/task_analysis_surface/visualization/final_cifti_figures/task-{task}_contrast1-{contrasts[0]}_contrast2-{contrasts[1]}_contrast3-{contrasts[2]}_cifti_figure.png')

    return

In [ ]:
groups=['HC','MM']
sessions=['baseline','1year']
task='mid'


if task == 'mid':
    contrasts=['HiWin+LoWin-HiNoWin-LoNoWin','HiLoss+LoLoss-AvoidHiLoss-AvoidLoLoss']

elif task == 'sst':
    contrasts=['SuccStop-Go','UnsuccStop-Go','UnsuccStop-SuccStop']

elif task == 'nback':
    contrasts=['twoback-zeroback']




for contrast in contrasts:
    for group in groups:
        for ses in sessions:
            
            create_surf_figure_by_group(group,ses,task,contrast)
            create_subcortical_figure_by_group(group,ses,task,contrast)
            height_surf, height_subcortical = create_cifti_figure_by_group(group,ses,task,contrast)
    
    width_surf = create_cifti_figure_by_contrast(task, contrast)

    
create_surf_figure_all(task,contrasts,width_surf,height_surf,height_subcortical) 